In [ ]:
# '''
# Data download
# '''
# !wget https://zenodo.org/record/1489920/files/articles-training-bypublisher-20181122.zip?download=1

# !wget https://zenodo.org/record/1489920/files/ground-truth-training-bypublisher-20181122.zip?download=1

# !mv 'articles-training-bypublisher-20181122.zip?download=1' bypublisher.zip
# !mv 'ground-truth-training-bypublisher-20181122.zip?download=1' gd.zip
# !unzip gd.zip
# !unzip bypublisher.zip
# !mv 'articles-training-bypublisher-20181122.xml' bypublisher.xml
# !mv 'ground-truth-training-bypublisher-20181122.xml' gd.xml
# !rm bypublisher.zip
# !rm gd.zip

In [ ]:
!pip install transformers
import pandas as pd
import numpy as np
import os
import sys
import xml.sax
import xml.parsers.expat
import time 
import pickle
import os
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer 
from collections import OrderedDict
import re

In [3]:
# Data Loader
titles = []
labels = []
text=[]

class ParseHandler(xml.sax.handler.ContentHandler):
    def __init__(self):
        self.text=""
    def startElement(self, name, attrs):
        self.CurrentData = name
        if(self.CurrentData=="article"):
            labels.append(attrs['hyperpartisan'])


parser = xml.sax.make_parser()
parser.setFeature(xml.sax.handler.feature_namespaces, 0) 
Handler = ParseHandler()
parser.setContentHandler(Handler)
parser.parse("gd.xml")


class ParseHandler1(xml.sax.handler.ContentHandler):
    def __init__(self):
        self.text=""
        self.r1=0
    def startElement(self, name, attrs):
        self.CurrentData = name
        if 'title' in attrs.getNames():
            titles.append(attrs.getValue('title'))
    def endElement(self, name):
        if(name=="article"):
            self.text=clean_text(self.text)
            self.text=re.sub(r'([^\s\w]|_)+', '', self.text)
            self.text=self.text.strip()
            text.append(self.text)
            self.text=""    
    def characters(self, data):
        self.text += data

def clean_text(text1):
    text1 = text1.replace(".", ". ")
    text1 = text1.replace(" _", " ")
    text1 = text1.replace("  ", " ")
    text1 = text1.replace(". . . .", "...")
    text1 = text1.replace(". . .", "...")
    return text1

parser = xml.sax.make_parser()
parser.setFeature(xml.sax.handler.feature_namespaces, 0) 
Handler = ParseHandler1()
parser.setContentHandler(Handler)
parser.parse("bypublisher.xml")


df_text = pd.DataFrame(text)
df_label = pd.DataFrame(labels)
df_title = pd.DataFrame(titles)
# print(df1)
result = pd.concat([df_text, df_title, df_label], axis=1, sort=False) 
result.columns = ['texts','title','labels']
result.to_csv('publisher.csv')


In [2]:
# print(result.columns)
# !cp publisher.csv drive/My\ Drive/ANLP_proj/
!cp 'drive/My Drive/ANLP_proj/publisher.csv' .

In [11]:
max_articles = 10000
durl = 'drive/My Drive/ANLP_proj/'
articles = []
labels = []
titles = []
csv_File = 'publisher.csv'
# csv_File = durl + 'publisher.csv'
df_itr = pd.read_csv(csv_File,chunksize=100)

cur = 0
for it in df_itr:
  if cur >= max_articles//100:
    break
  articles.extend(it['texts'])
  labels.extend(it['labels'])
  titles.extend(it['title'])
  # break
  cur+=1
  # for row in it:
    # print(row)
    # articles.extend(row.iloc[cur]['texts'])
    # labels.extend(row.iloc[cur]['labels'])
labels = np.array(labels,dtype = np.uint8)
print('number of samples = ',len(labels),len(titles))

number of samples =  10000 10000


In [ ]:
# df = pd.read_csv('article.csv')
# articles = df['texts'].to_numpy()
# labels = df['labels'].to_numpy()
# print(articles.shape)
print(titles[0])
print('********************')
print(articles[0])

In [5]:
import tensorflow as tf
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
max_length_test = 50
def map_example_to_dict(input_ids_list_content, attention_mask_list_content, token_type_ids_list_content, 
                        input_ids_list_title, token_type_ids_list_title, attention_mask_list_title, label):
  return ({
      "input_ids": input_ids_list_content,
      "token_type_ids": attention_mask_list_content,
      "attention_mask": token_type_ids_list_content,
  },
  {
      "input_ids": input_ids_list_title,
      "token_type_ids": token_type_ids_list_title,
      "attention_mask": attention_mask_list_title,
  }, label)

def encode_examples(content,titles,labels):
  # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list_content = []
  token_type_ids_list_content = []
  attention_mask_list_content = []

  input_ids_list_title = []
  token_type_ids_list_title = []
  attention_mask_list_title = []

  label_list = []
  for article, title, label in zip(content, titles, labels):
    if not (isinstance(article, str) and isinstance(title, str)) :
      continue
    bert_input_content = tokenizer.encode_plus(article,
                                      truncation=True,           
                                      add_special_tokens = True, # add [CLS], [SEP]
                                      max_length = max_length_test, # max length of the text that can go to BERT
                                      padding='max_length', # add [PAD] tokens
                                      return_attention_mask = True, # add attention mask to not focus on pad token
                                      )


    bert_input_title = tokenizer.encode_plus(title,
                                      truncation=True,           
                                      add_special_tokens = True, # add [CLS], [SEP]
                                      max_length = max_length_test, # max length of the text that can go to BERT
                                      padding='max_length', # add [PAD] tokens
                                      return_attention_mask = True, # add attention mask to not focus on pad token
                                      )

    input_ids_list_content.append(bert_input_content['input_ids'])
    token_type_ids_list_content.append(bert_input_content['token_type_ids'])
    attention_mask_list_content.append(bert_input_content['attention_mask'])

    input_ids_list_title.append(bert_input_title['input_ids'])
    token_type_ids_list_title.append(bert_input_title['token_type_ids'])
    attention_mask_list_title.append(bert_input_title['attention_mask'])

    label_list.append([label])
  return tf.data.Dataset.from_tensor_slices((input_ids_list_content, attention_mask_list_content, token_type_ids_list_content, 
                                             input_ids_list_title, token_type_ids_list_title, attention_mask_list_title, label_list)
                                            ).map(map_example_to_dict)

def convert_to_tfdataset(articles,titles,labels,batch_size = 5):
  dataset = encode_examples(articles,titles,labels)
  dataset = dataset.shuffle(dataset.__len__().numpy()).batch(batch_size)
  return dataset


In [12]:
# split
from sklearn.model_selection import train_test_split
articles_train, articles_test, titles_train, titles_test, labels_train, labels_test = train_test_split( articles, titles, labels, test_size=0.3, random_state=21)
train_dataset = convert_to_tfdataset(articles_train,titles_train,labels_train,batch_size = 6)
test_dataset = convert_to_tfdataset(articles_test,titles_test,labels_test,batch_size = 6)

In [ ]:
for input in train_dataset:
  print(input)
  break

In [8]:
from transformers import BertConfig
from transformers import TFBertModel

model_name = 'bert-base-uncased'
config1 = BertConfig.from_pretrained(model_name)
config1.num_labels=2

model_name = 'bert-base-uncased'
bert_model = TFBertModel.from_pretrained(model_name,config = config1)
learning_rate = 2e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
bert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# bert_history = bert_model.fit(train_dataset,epochs=5,verbose=1,validation_data=test_dataset)
# print(tokenizer("hello there",return_tensors="tf"))


In [ ]:
# class MyClassifier(tf.keras.Model):
#   def __init__(self,N_LABELS):
#     super(MyClassifier, self).__init__()
#     self.bert = bert_model
#     self.fc = tf.keras.layers.Dense(N_LABELS,activation='tanh',name='my_fc_layer')

#   def call(self, input):
#     # mask = tf.sequence_mask(inputs[1], tf.shape(inputs[0])[1])
#     bert_embeddings = self.bert(input)
#     output = self.fc(bert_embeddings[1]) #only last hidden layer
#     return output

# class Trainer:
#   def __init__(self,N_LABELS):
#     super(Trainer, self).__init__()
#     self.model = MyClassifier(N_LABELS)

#   def lossfn(self, predictions, labels):      
#     scce = tf.keras.losses.SparseCategoricalCrossentropy()
#     loss = scce(labels,predictions)
#     return loss

#   def train(self,dataset,epochs):
#     optimizer = tf.keras.optimizers.SGD(learning_rate= 1e-3)
#     # logdir="logs"
#     # train_writer = tf.summary.create_file_writer(logdir+'/train')
#     # test_writer = tf.summary.create_file_writer(logdir+'/test')
#     for epoch in range(epochs):
#       losses = []
#       for batch,(inputs, labels) in enumerate(dataset):
#         with tf.GradientTape() as tape:
#           predictions = self.model(inputs)
#           loss_value = self.lossfn(predictions,labels)
#         losses.append(loss_value)
#         tvars = self.model.trainable_variables
#         # tvars = [v for v in tvars if 'bert' not in v.name]
#         # print('this',tvars)
#         grads = tape.gradient(loss_value, tvars)
#         optimizer.apply_gradients(zip(grads, tvars))
#         # predictedLabels, score = self.decode(emission, transition_params, seqLength)
#       print(np.mean(losses))

# n_labels = 2
# classifier = Trainer(n_labels)
# classifier.train(train_dataset,epochs = 5)

In [ ]:
class MyClassifier(tf.keras.Model):
  def __init__(self,N_LABELS):
    super(MyClassifier, self).__init__()
    self.bert = bert_model
    self.fc1 = tf.keras.layers.Dense(32,activation='tanh',name='my_fc_layer1')
    self.fc2 = tf.keras.layers.Dense(N_LABELS,activation='tanh',name='my_fc_layer2')

  def call(self, content, title):
    # mask = tf.sequence_mask(inputs[1], tf.shape(inputs[0])[1])
    # content, title = inputs
    bert_embedding_content = self.bert(content)
    bert_embedding_title = self.bert(title)
    # print(bert_embedding_content)
    # print('************************')
    # print(bert_embedding_title)
    concatenated = tf.concat([bert_embedding_content[1],bert_embedding_title[1]], axis = -1 )
    output = self.fc1( self.fc2(concatenated) ) #only last hidden layer
    return output

class Trainer:
  def __init__(self,N_LABELS):
    super(Trainer, self).__init__()
    self.model = MyClassifier(N_LABELS)
    self.accuracy_metric = tf.keras.metrics.Accuracy()

  def lossfn(self, predictions, labels):      
    scce = tf.keras.losses.SparseCategoricalCrossentropy()
    loss = scce(labels,predictions)
    return loss

  def train(self,dataset,epochs):
    optimizer = tf.keras.optimizers.SGD(learning_rate= 1e-3)
    # logdir="logs"
    # train_writer = tf.summary.create_file_writer(logdir+'/train')
    # test_writer = tf.summary.create_file_writer(logdir+'/test')
    for epoch in range(epochs):
      losses = []
      for batch,(contents, titles, labels) in enumerate(dataset):
        with tf.GradientTape() as tape:
          predictions = self.model(contents, titles)
          loss_value = self.lossfn(predictions,labels)
        losses.append(loss_value)
        tvars = self.model.trainable_variables
        tvars = [v for v in tvars if 'bert' not in v.name]
        # print('this',tvars)
        grads = tape.gradient(loss_value, tvars)
        optimizer.apply_gradients(zip(grads, tvars))
        # predictedLabels, score = self.decode(emission, transition_params, seqLength)
      print(np.mean(losses))

  def evaluate(self,dataset):
    for batch,(contents, titles, labels) in enumerate(dataset):
      
      predictions = self.model(contents, titles)
      predictions = np.argmax(predictions,axis=1)
      self.accuracy_metric.update_state(predictions, labels)
      # print(predictions)
      print(self.accuracy_metric.result().numpy())


n_labels = 2
classifier = Trainer(n_labels)
classifier.train(train_dataset_publisher,epochs = 3)
classifier.train(train_dataset_article,epochs = 3)
classifier.evaluate(test_dataset_publisher)
classifier.evaluate(test_dataset_article)